In [253]:
from tqdm import tqdm
from convokit import Corpus, Speaker, Utterance
from collections import defaultdict

In [254]:
# replace the directory with where your downloaded cornell movie dialogs corpus is saved
data_dir = "C:\\Users\\Vinay\\OneDrive\\Documents\\GitHub\\Coordination\\ijcnlp_dailydialog\\"

In [255]:
with open(data_dir + "dialogues_text.txt", "r", encoding='utf-8', errors='ignore') as f:
    dialog_data = f.readlines()

In [256]:
utterance_data=[]
count=0
for utterance in tqdm(dialog_data):
    
    utterance_data.append( [info.strip( ) for info in utterance.split(" __eou__")])

100%|██████████| 13118/13118 [00:00<00:00, 253076.61it/s]


In [257]:

for i in range(len(utterance_data)):
    utterance_data[i].pop()

In [258]:
#adding utterance index for a dialog
x=0
for i in range(len(utterance_data)):
    for j in range(len(utterance_data[i])):
        utterance_data[i][j]+=" __alp__ "+(str(j+x))
    x+=len(utterance_data[i])
        

In [259]:
with open(data_dir + "dialogues_act.txt", "r", encoding='utf-8', errors='ignore') as f:
    dialog_act = f.readlines()

In [260]:
utterance_act=[]
for utterance in tqdm(dialog_act):
    
    utterance_act.append([info.strip() for info in utterance.split(" ")])

100%|██████████| 13118/13118 [00:00<00:00, 416553.46it/s]


In [261]:
for i in range(len(utterance_data)):
    for j in range(len(utterance_data[i])):
        utterance_data[i][j]+=" __alp__ "+utterance_act[i][j]

In [262]:
with open(data_dir + "dialogues_emotion.txt", "r", encoding='utf-8', errors='ignore') as f:
    dialog_emotion = f.readlines()

In [263]:
utterance_emotion=[]
for utterance in tqdm(dialog_emotion):
    
    utterance_emotion .append( [info.strip() for info in utterance.split(" ")])

100%|██████████| 13118/13118 [00:00<00:00, 340851.18it/s]


In [264]:
#emotion index
for i in range(len(utterance_data)):
    for j in range(len(utterance_data[i])):
        utterance_data[i][j]+=" __alp__ "+utterance_emotion[i][j]

In [265]:
utterance_data[1]

['So Dick , how about getting some coffee for tonight ? __alp__ 2 __alp__ 3 __alp__ 4',
 'Coffee ? I don ’ t honestly like that kind of stuff . __alp__ 3 __alp__ 4 __alp__ 2',
 'Come on , you can at least try a little , besides your cigarette . __alp__ 4 __alp__ 3 __alp__ 0',
 'What ’ s wrong with that ? Cigarette is the thing I go crazy for . __alp__ 5 __alp__ 1 __alp__ 1',
 'Not for me , Dick . __alp__ 6 __alp__ 1 __alp__ 0']

In [266]:
#adding speakers
#we will assume speaker 0 and 1 speaking alternatively
for i in range(len(utterance_data)):
    for j in range(len(utterance_data[i])):
        utterance_data[i][j]+=" __alp__ "+str(2*i+j%2)
utterance_data[1]


['So Dick , how about getting some coffee for tonight ? __alp__ 2 __alp__ 3 __alp__ 4 __alp__ 2',
 'Coffee ? I don ’ t honestly like that kind of stuff . __alp__ 3 __alp__ 4 __alp__ 2 __alp__ 3',
 'Come on , you can at least try a little , besides your cigarette . __alp__ 4 __alp__ 3 __alp__ 0 __alp__ 2',
 'What ’ s wrong with that ? Cigarette is the thing I go crazy for . __alp__ 5 __alp__ 1 __alp__ 1 __alp__ 3',
 'Not for me , Dick . __alp__ 6 __alp__ 1 __alp__ 0 __alp__ 2']

In [267]:
#conversation id
for i in range(len(utterance_data)):
    for j in range(len(utterance_data[i])):
        utterance_data[i][j]+=" __alp__ "+str(i)

In [268]:
utterance_data[1]

['So Dick , how about getting some coffee for tonight ? __alp__ 2 __alp__ 3 __alp__ 4 __alp__ 2 __alp__ 1',
 'Coffee ? I don ’ t honestly like that kind of stuff . __alp__ 3 __alp__ 4 __alp__ 2 __alp__ 3 __alp__ 1',
 'Come on , you can at least try a little , besides your cigarette . __alp__ 4 __alp__ 3 __alp__ 0 __alp__ 2 __alp__ 1',
 'What ’ s wrong with that ? Cigarette is the thing I go crazy for . __alp__ 5 __alp__ 1 __alp__ 1 __alp__ 3 __alp__ 1',
 'Not for me , Dick . __alp__ 6 __alp__ 1 __alp__ 0 __alp__ 2 __alp__ 1']

In [269]:
utterance_data_new=[]
for xs in utterance_data:
    for x in xs:
        utterance_data_new.append(x)
utterance_data_new[-2]

"Ok , I'll get this rate right away . __alp__ 102978 __alp__ 4 __alp__ 0 __alp__ 26235 __alp__ 13117"

In [270]:
speaker_meta={}
for i in range(26236):
    speaker_meta[str(i)]={"speaker_id": i}

In [271]:
corpus_speakers = {k: Speaker(id = k, meta = v) for k,v in speaker_meta.items()}

In [272]:
utterance_corpus = {}
count = 0
for utterance in tqdm(utterance_data_new):
    
    utterance_info = [info.strip() for info in utterance.split(" __alp__")]
    
    if len(utterance_info) < 4:
        print(utterance_info)
        
    try:
        text, utt_idx, utt_act, utt_emo,speaker_id,conver_id = utterance_info[0], utterance_info[1], utterance_info[2], int(utterance_info[3]),utterance_info[4],utterance_info[5]
    except:
        print(utterance_info)
    
    meta = {'utt_emo': utt_emo, 'utt_idx': utt_idx,'utt_act': utt_act}
    
    # root & reply_to will be updated later, timestamp is not applicable 
    utterance_corpus[utt_idx] = Utterance(id=utt_idx, speaker= corpus_speakers[speaker_id], text=text, conversation_id=conver_id, meta= meta)

print("Total number of utterances = {}".format(len(utterance_corpus)))

100%|██████████| 102980/102980 [00:01<00:00, 72753.92it/s]

Total number of utterances = 102980


In [273]:
utterance_corpus['0']

Utterance({'obj_type': 'utterance', 'meta': {'utt_emo': 2, 'utt_idx': '0', 'utt_act': '3'}, 'vectors': [], 'speaker': Speaker({'obj_type': 'speaker', 'meta': {'speaker_id': 0}, 'vectors': [], 'owner': None, 'id': '0'}), 'conversation_id': '0', 'reply_to': None, 'timestamp': None, 'text': 'The kitchen stinks .', 'owner': None, 'id': '0'})

In [274]:
len(utterance_corpus)

102980

In [275]:
conv_id_definer=0
for i in range(len(utterance_corpus)):
    if i==0 or utterance_corpus[str(i)].conversation_id!=utterance_corpus[str(i-1)].conversation_id:
        utterance_corpus[str(i)].reply_to=None
    else:
        utterance_corpus[str(i)].reply_to=str(i-1)


In [276]:
print(utterance_corpus['20'])
print(utterance_corpus['50'].conversation_id)
print(utterance_corpus['50'].speaker.meta)


Utterance(id: '20', conversation_id: 5, reply-to: 19, speaker: Speaker(id: 10, vectors: [], meta: {'speaker_id': 10}), timestamp: None, text: 'What ’ s wrong ? Didn ’ t you think it was fun ? !', vectors: [], meta: {'utt_emo': 6, 'utt_idx': '20', 'utt_act': '2'})
10
{'speaker_id': 21}


In [277]:
utternace_list=utterance_corpus.values()

In [278]:
coordination_corpus=Corpus(utterances=utternace_list)

In [279]:
print("number of conversations in the dataset = {}".format(len(coordination_corpus.get_conversation_ids())))

number of conversations in the dataset = 13118


In [280]:
convo_ids = coordination_corpus.get_conversation_ids()
for i, convo_idx in enumerate(convo_ids[0:4]):
    print("sample conversation {}:".format(i))
    print(coordination_corpus.get_conversation(convo_idx).get_utterance_ids())

sample conversation 0:
['0', '1']
sample conversation 1:
['2', '3', '4', '5', '6']
sample conversation 2:
['7', '8', '9', '10']
sample conversation 3:
['11', '12', '13', '14']


In [281]:
coordination_corpus.dump("coordination-corpus")

In [282]:
from convokit import meta_index
import os.path

In [283]:
meta_index(filename = os.path.join(os.path.expanduser("~"), ".convokit/saved-corpora/coordination-corpus"))

{'utterances-index': {'utt_emo': ["<class 'int'>"],
  'utt_idx': ["<class 'str'>"],
  'utt_act': ["<class 'str'>"]},
 'speakers-index': {'speaker_id': ["<class 'int'>"]},
 'conversations-index': {},
 'overall-index': {},
 'version': 1,
 'vectors': []}

In [284]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import convokit
%matplotlib inline

In [285]:
corpus= Corpus(utterances=utternace_list)

In [286]:
print(corpus)

In [287]:
coord= convokit.Coordination()
coord.fit(corpus)
coord.transform(corpus)





In [288]:
dictionary_of_emotions={}
list_of_emotion=[]
for i in range(len(utterance_corpus)):
    emotion_key=utterance_corpus[str(i)].speaker.meta['speaker_id']
    emotion_dict=utterance_corpus[str(i)].meta['utt_emo']
    if emotion_key in dictionary_of_emotions:
        dictionary_of_emotions[emotion_key].append(emotion_dict)
    else: 
        dictionary_of_emotions[emotion_key]=[emotion_dict]
for i in range(26236):
    list_of_emotion.append(dictionary_of_emotions[i])

In [346]:
#a to b
#initial= not equal
#final=equal
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
t=set()
q=set()
speakera_to_speakerb=coord.summarize(corpus, speaker_a, speaker_b)
for speakera, score in sorted(speakera_to_speakerb.averages_by_speaker().items(),
    key=lambda x: x[1], reverse=True):
    if(round(score,5)==0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)-1]
    if emotion_speaker1[0]!=emotion_speaker2[0]:
        t.add(int(speakera.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
    #if (emotion_speaker1[0]!=emotion_speaker2[0]) and (((emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0))):
        #t.discard(int(speakera.id))
    if length1>1:
        if emotion_speaker1[0]!=emotion_speaker2[0] and emotion_speaker1[length1-1]==emotion_speaker2[length2-1]:
            q.add(int(speakera.id))
        #if (emotion_speaker1[0]!=emotion_speaker2[0]) and (((emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0))):
            #q.discard(int(speakera.id))
len(t)




Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:


33

In [347]:
#b to a 
#initial= not equal
#final= equal
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
speakerb_to_speakera=coord.summarize(corpus, speaker_b, speaker_a)
for speakerb, score in sorted(speakerb_to_speakera.averages_by_speaker().items(),key=lambda x: x[1], reverse=True):
    if(round(score,5)==0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)-1]
    if emotion_speaker1[0]!=emotion_speaker2[0]:
        t.add(int(speakerb.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
   # if (emotion_speaker1[0]!=emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
        #t.discard(int(speakerb.id))
    if length1>1:
        if emotion_speaker1[0]!=emotion_speaker2[0] and emotion_speaker1[length1-1]==emotion_speaker2[length2-1]:
            q.add(int(speakerb.id))
        #if (emotion_speaker1[0]!=emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
            #q.discard(int(speakerb.id))
print(len(t),len(q)/len(t))

Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:
115 0.4


In [348]:
print("Thus, the total number percentage of cases in which the eventual(final) emotion of 2 speakers whose initial emotions are not equal are ",100*len(q)/len(t))

Thus, the total number percentage of cases in which the eventual(final) emotion of 2 speakers whose initial emotions are not equal are  40.0


In [349]:
#a to b
#initial= equal
#final= equal
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
w=set()
e=set()
speakera_to_speakerb=coord.summarize(corpus, speaker_a, speaker_b)
for speakera, score in sorted(speakera_to_speakerb.averages_by_speaker().items(),
    key=lambda x: x[1], reverse=True):
    if(round(score,5)==0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)-1]
    if emotion_speaker1[0]==emotion_speaker2[0]:
        w.add(int(speakera.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
    #if (emotion_speaker1[0]==emotion_speaker2[0]) or (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
        #w.discard(int(speakera.id))
    if length1>1:
        if emotion_speaker1[0]==emotion_speaker2[0] and emotion_speaker1[length1-1]==emotion_speaker2[length2-1]:
            e.add(int(speakera.id))
        #if (emotion_speaker1[0]==emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
            #e.discard(int(speakera.id))
len(w)




Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:


273

In [350]:
#b to a 
#initial= equal
#final= equal
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
speakerb_to_speakera=coord.summarize(corpus, speaker_b, speaker_a)
for speakerb, score in sorted(speakerb_to_speakera.averages_by_speaker().items(),key=lambda x: x[1], reverse=True):
    if(round(score,5)==0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)-1]
    if emotion_speaker1[0]==emotion_speaker2[0]:
        w.add(int(speakerb.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
    #if (emotion_speaker1[0]==emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
        #w.discard(int(speakerb.id))

    if length1>1:
        if emotion_speaker1[0]==emotion_speaker2[0] and emotion_speaker1[length1-1]==emotion_speaker2[length2-1]:
            e.add(int(speakerb.id))
        #if (emotion_speaker1[0]==emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
            #e.discard(int(speakerb.id))
print(len(w),len(e)/len(w))

Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:
651 0.6605222734254992


In [351]:
print("Thus, the total number percentage of cases in which the eventual(final) emotion of 2 speakers whose initial emotions are also same are ",100*len(e)/len(w))

Thus, the total number percentage of cases in which the eventual(final) emotion of 2 speakers whose initial emotions are also same are  66.05222734254993


In [352]:
#a to b
#initial= not equal
#final= similar
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
similar_t=set()
similar_q=set()
speakera_to_speakerb=coord.summarize(corpus, speaker_a, speaker_b)
for speakera, score in sorted(speakera_to_speakerb.averages_by_speaker().items(),
    key=lambda x: x[1], reverse=True):
    if(round(score,5)==0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)-1]
    if emotion_speaker1[0]!=emotion_speaker2[0]:
        similar_t.add(int(speakera.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
    #if (emotion_speaker1[0]!=emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
        #similar_t.discard(int(speakera.id))
    if length1>1:
        if emotion_speaker1[0]!=emotion_speaker2[0]:
            if emotion_speaker1[length1-1]==emotion_speaker2[length2-1] or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==2) or (emotion_speaker1[length1-1]==2 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==2 and emotion_speaker2[length2-1]==3) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==2) or (emotion_speaker1[length1-1]==4 and emotion_speaker2[length2-1]==6) or (emotion_speaker1[length1-1]==6 and emotion_speaker2[length2-1]==4) or (emotion_speaker1[length1-1]==6 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==6) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==5) or (emotion_speaker1[length1-1]==5 and emotion_speaker2[length2-1]==3) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==5) or (emotion_speaker1[length1-1]==5 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==3) :
                similar_q.add(int(speakera.id))
            #if (emotion_speaker1[0]!=emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
                #similar_q.discard(int(speakera.id))
len(similar_t)


Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:


33

In [353]:
#b to a 
#initial= not equal
#final= similar
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
speakerb_to_speakera=coord.summarize(corpus, speaker_b, speaker_a)
for speakerb, score in sorted(speakerb_to_speakera.averages_by_speaker().items(),key=lambda x: x[1], reverse=True):
    if(round(score,5)==0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)-1]
    if emotion_speaker1[0]!=emotion_speaker2[0]:
        similar_t.add(int(speakerb.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
    #if (emotion_speaker1[0]!=emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
        #similar_t.discard(int(speakerb.id))
    if length1>1:
        if emotion_speaker1[0]!=emotion_speaker2[0]:
            if emotion_speaker1[length1-1]==emotion_speaker2[length2-1] or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==2) or (emotion_speaker1[length1-1]==2 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==2 and emotion_speaker2[length2-1]==3) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==2) or (emotion_speaker1[length1-1]==4 and emotion_speaker2[length2-1]==6) or (emotion_speaker1[length1-1]==6 and emotion_speaker2[length2-1]==4) or (emotion_speaker1[length1-1]==6 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==6) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==5) or (emotion_speaker1[length1-1]==5 and emotion_speaker2[length2-1]==3) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==5) or (emotion_speaker1[length1-1]==5 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==3)  :
                similar_q.add(int(speakerb.id))
            #if (emotion_speaker1[0]!=emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
                #similar_q.discard(int(speakerb.id))
print(len(similar_t),len(similar_q)/len(similar_t))

Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:
115 0.41739130434782606


In [354]:
print("Thus, the total number percentage of cases in which the eventual(final) emotion of 2 speakers is similar whose initial emotions are not same are are ",100*len(similar_q)/len(similar_t))

Thus, the total number percentage of cases in which the eventual(final) emotion of 2 speakers is similar whose initial emotions are not same are are  41.73913043478261


In [355]:
#a to b
#initial= equal
#final= similar
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
similar_t_equal=set()
similar_q_equal=set()
speakera_to_speakerb=coord.summarize(corpus, speaker_a, speaker_b)
for speakera, score in sorted(speakera_to_speakerb.averages_by_speaker().items(),
    key=lambda x: x[1], reverse=True):
    if(round(score,5)==0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)-1]
    if emotion_speaker1[0]==emotion_speaker2[0]:
        similar_t_equal.add(int(speakera.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
    #if (emotion_speaker1[0]==emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
        #similar_t_equal.discard(int(speakera.id))
    if length1>1:
        if emotion_speaker1[0]==emotion_speaker2[0]:
            if emotion_speaker1[length1-1]==emotion_speaker2[length2-1] or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==2) or (emotion_speaker1[length1-1]==2 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==2 and emotion_speaker2[length2-1]==3) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==2) or (emotion_speaker1[length1-1]==4 and emotion_speaker2[length2-1]==6) or (emotion_speaker1[length1-1]==6 and emotion_speaker2[length2-1]==4) or (emotion_speaker1[length1-1]==6 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==6) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==5) or (emotion_speaker1[length1-1]==5 and emotion_speaker2[length2-1]==3) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==5) or (emotion_speaker1[length1-1]==5 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==3) :
                similar_q_equal.add(int(speakera.id))
            #if (emotion_speaker1[0]==emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
                #similar_q_equal.discard(int(speakera.id))
len(similar_t_equal)

Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:


273

In [356]:
#b to a 
#initial= equal
#final= similar
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
speakerb_to_speakera=coord.summarize(corpus, speaker_b, speaker_a)
for speakerb, score in sorted(speakerb_to_speakera.averages_by_speaker().items(),key=lambda x: x[1], reverse=True):
    if(round(score,5)==0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)-1]
    if emotion_speaker1[0]==emotion_speaker2[0]:
        similar_t_equal.add(int(speakerb.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
    #if (emotion_speaker1[0]==emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
        #similar_t_equal.discard(int(speakerb.id))
    if length1>1:
        if emotion_speaker1[0]==emotion_speaker2[0]:
            if emotion_speaker1[length1-1]==emotion_speaker2[length2-1] or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==2) or (emotion_speaker1[length1-1]==2 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==2 and emotion_speaker2[length2-1]==3) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==2) or (emotion_speaker1[length1-1]==4 and emotion_speaker2[length2-1]==6) or (emotion_speaker1[length1-1]==6 and emotion_speaker2[length2-1]==4) or (emotion_speaker1[length1-1]==6 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==6) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==5) or (emotion_speaker1[length1-1]==5 and emotion_speaker2[length2-1]==3) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==5) or (emotion_speaker1[length1-1]==5 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==3 and emotion_speaker2[length2-1]==1) or (emotion_speaker1[length1-1]==1 and emotion_speaker2[length2-1]==3)  :
                similar_q_equal.add(int(speakerb.id))
            #if (emotion_speaker1[0]==emotion_speaker2[0]) and (emotion_speaker1[0]==0 and emotion_speaker1[length1-1]==0) or (emotion_speaker2[0]==0 and emotion_speaker2[length2-1]==0):
                #similar_q_equal.discard(int(speakerb.id))
print(len(similar_t_equal),len(similar_q_equal)/len(similar_t_equal))

Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:
651 0.6666666666666666


In [357]:
print("Thus, the total number percentage of cases in which the eventual(final) emotion of 2 speakers is similar whose initial emotions are also same are are ",100*len(similar_q_equal)/len(similar_t_equal))

Thus, the total number percentage of cases in which the eventual(final) emotion of 2 speakers is similar whose initial emotions are also same are are  66.66666666666667


In [358]:
#Hypothesis 2

In [359]:
#a to b
#initial= any emotion
#final= negative emotion of any one speaker
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
hypo2_t=set()
hypo2_q=set()
speakera_to_speakerb=coord.summarize(corpus, speaker_a, speaker_b)
for speakera, score in sorted(speakera_to_speakerb.averages_by_speaker().items(),
    key=lambda x: x[1], reverse=False):
    if(round(score,5)==-0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)-1]
    hypo2_t.add(int(speakera.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
    if length1>1:
        if ((emotion_speaker1[length1-1]==0) or (emotion_speaker1[length1-1]==1) or (emotion_speaker1[length1-1]==2) or (emotion_speaker1[length1-1]==3) or (emotion_speaker1[length1-1]==5)) and ((emotion_speaker2[length2-1]==0) or emotion_speaker2[length2-1]==1 or emotion_speaker2[length2-1]==2 or emotion_speaker2[length2-1]==3 or emotion_speaker2[length2-1]==5):
            hypo2_q.add(int(speakera.id))
len(hypo2_t)




Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:


255

In [360]:
#b to a 
#initial= anything
#final= negative emotion of any one speaker
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
speakerb_to_speakera=coord.summarize(corpus, speaker_b, speaker_a)
for speakerb, score in sorted(speakerb_to_speakera.averages_by_speaker().items(),key=lambda x: x[1], reverse=False):
    if(round(score,5)==-0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)-1]
    hypo2_t.add(int(speakerb.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
    if length1>1:
        if (emotion_speaker1[length1-1]==0 or emotion_speaker1[length1-1]==1 or emotion_speaker1[length1-1]==2 or emotion_speaker1[length1-1]==3 or emotion_speaker1[length1-1]==5) and (emotion_speaker2[length2-1]==0 or emotion_speaker2[length2-1]==1 or emotion_speaker2[length2-1]==2 or emotion_speaker2[length2-1]==3 or emotion_speaker2[length2-1]==5):
            hypo2_q.add(int(speakerb.id))
print(len(hypo2_t),len(hypo2_q)/len(hypo2_t))

Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:
496 0.5705645161290323


In [361]:
print("Thus, the total number percentage of cases in which the eventual(final) emotion of both 2 speakers is negative or no emotion are ",100*len(hypo2_q)/len(hypo2_t))

Thus, the total number percentage of cases in which the eventual(final) emotion of both 2 speakers is negative or no emotion are  57.056451612903224


In [362]:
#a to b
#initial= not negative emotion for both speakers
#final= negative emotion of any one speaker
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
hypo2_t_ini=set()
hypo2_q_ini=set()
speakera_to_speakerb=coord.summarize(corpus, speaker_a, speaker_b)
for speakera, score in sorted(speakera_to_speakerb.averages_by_speaker().items(),
    key=lambda x: x[1], reverse=False):
    if(round(score,5)==-0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakera.id)]
        emotion_speaker2=list_of_emotion[int(speakera.id)-1]
    if (emotion_speaker1[0]==0 or emotion_speaker1[0]==4 or emotion_speaker1[0]==6) and (emotion_speaker2[0]==0 or emotion_speaker2[0]==4 or emotion_speaker2[0]==6):
        hypo2_t_ini.add(int(speakera.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
    if length1>1:
        if ((emotion_speaker1[0]==0 or emotion_speaker1[0]==4 or emotion_speaker1[0]==6) and (emotion_speaker2[0]==0 or emotion_speaker2[0]==4 or emotion_speaker2[0]==6)) and (((emotion_speaker1[length1-1]==0) or (emotion_speaker1[length1-1]==1) or (emotion_speaker1[length1-1]==2) or (emotion_speaker1[length1-1]==3) or (emotion_speaker1[length1-1]==5)) and ((emotion_speaker2[length2-1]==0) or emotion_speaker2[length2-1]==1 or emotion_speaker2[length2-1]==2 or emotion_speaker2[length2-1]==3 or emotion_speaker2[length2-1]==5)):
            hypo2_q_ini.add(int(speakera.id))
len(hypo2_t_ini)




Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:


242

In [363]:
#b to a 
#initial= not negative emotion for both speakers
#final= negative emotion of any one speaker
print('Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:')
speakerb_to_speakera=coord.summarize(corpus, speaker_b, speaker_a)
for speakerb, score in sorted(speakerb_to_speakera.averages_by_speaker().items(),key=lambda x: x[1], reverse=False):
    if(round(score,5)==-0.1):
        break
    if int(speakera.id)%2==0:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)+1]
    else:
        emotion_speaker1=list_of_emotion[int(speakerb.id)]
        emotion_speaker2=list_of_emotion[int(speakerb.id)-1]
    if (emotion_speaker1[0]==0 or emotion_speaker1[0]==4 or emotion_speaker1[0]==6) and (emotion_speaker2[0]==0 or emotion_speaker2[0]==4 or emotion_speaker2[0]==6):
        hypo2_t_ini.add(int(speakerb.id))
    length1=len(emotion_speaker1)
    length2=len(emotion_speaker2)
    if length1>1:
        if ((emotion_speaker1[0]==0 or emotion_speaker1[0]==4 or emotion_speaker1[0]==6) and (emotion_speaker2[0]==0 or emotion_speaker2[0]==4 or emotion_speaker2[0]==6)) and(((emotion_speaker1[length1-1]==0) or (emotion_speaker1[length1-1]==1) or (emotion_speaker1[length1-1]==2) or (emotion_speaker1[length1-1]==3) or (emotion_speaker1[length1-1]==5)) and ((emotion_speaker2[length2-1]==0) or emotion_speaker2[length2-1]==1 or emotion_speaker2[length2-1]==2 or emotion_speaker2[length2-1]==3 or emotion_speaker2[length2-1]==5)):
            hypo2_q_ini.add(int(speakerb.id))
print(len(hypo2_t_ini),len(hypo2_q_ini)/len(hypo2_t_ini))

Eventual emotion is same by what ratio if coordination of any one of the speakers is greater then 0.1 towards the other speaker:
475 0.5663157894736842


In [364]:
print("Thus, the total number percentage of cases in which the eventual(final) emotion of both 2 speakers is negative or no emotion and whose initial emotions are positive or no emotion are ",100*len(hypo2_q_ini)/len(hypo2_t_ini))

Thus, the total number percentage of cases in which the eventual(final) emotion of both 2 speakers is negative or no emotion and whose initial emotions are positive or no emotion are  56.63157894736842


In [ ]:
def make_chart(a_scores, b_scores, a_description, b_description, a_color="b", b_color="g"):
    # get scores by marker and on aggregate
    a_score_by_marker = a_scores["marker_agg2"]
    a_agg1, a_agg2, a_agg3 = a_scores["agg1"], a_scores["agg2"], a_scores["agg3"]
    b_score_by_marker = b_scores["marker_agg2"]
    b_agg1, b_agg2, b_agg3 = b_scores["agg1"], b_scores["agg2"], b_scores["agg3"]

    # the rest plots this data as a double bar graph
    a_data_points = sorted(a_score_by_marker.items())
    b_data_points = sorted(b_score_by_marker.items())
    a_data_points, b_data_points = zip(*sorted(zip(a_data_points, b_data_points),
        key=lambda x: x[0][1], reverse=True))
    labels, a_data_points = zip(*a_data_points)
    _, b_data_points = zip(*b_data_points)

    labels = ["aggregate 1", "aggregate 2", "aggregate 3"] + list(labels)
    a_data_points = [a_agg1, a_agg2, a_agg3] + list(a_data_points)
    b_data_points = [b_agg1, b_agg2, b_agg3] + list(b_data_points)

    fig, ax = plt.subplots()
    ax.set_xticks(np.arange(len(a_data_points)) + 0.35)
    ax.set_xticklabels(labels, rotation="vertical")

    ax.bar(np.arange(len(a_data_points)), a_data_points, 0.35, color=a_color)
    ax.bar(np.arange(len(b_data_points)) + 0.35, b_data_points, 0.35, color=b_color)

    b_patch = mpatches.Patch(color="b",
                             label=a_description + " (total: " +
                             str(a_scores["count_agg1"]) + ", " +
                             str(a_scores["count_agg2"]) + ")")
    g_patch = mpatches.Patch(color="g",
                             label=b_description + " (total: "  +
                             str(b_scores["count_agg1"]) + ", " +
                             str(b_scores["count_agg2"]) + ")")
    plt.legend(handles=[b_patch, g_patch])

    filename = str(a_description) + " vs " + str(b_description) + ".png"
    plt.savefig(filename, bbox_inches="tight")
    print('Created chart "' + filename + '"')